# Uploading Chunks to Vector Database

In [1]:
%pip install dbconnection -q

  error: subprocess-exited-with-error
  
  × Building wheel for oracledb (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [310 lines of output]
      /private/var/folders/ss/ytdznf2d6mxcx8d7tp5l1qb40000gn/T/pip-build-env-q96r4rwc/overlay/lib/python3.13/site-packages/setuptools/config/expand.py:128: SetuptoolsWarning: File '/private/var/folders/ss/ytdznf2d6mxcx8d7tp5l1qb40000gn/T/pip-install-u_1zbg7q/oracledb_994c40fe95d04017ae9e6f6950a26607/README.md' cannot be found
        for path in _filter_existing_files(_filepaths)
      /private/var/folders/ss/ytdznf2d6mxcx8d7tp5l1qb40000gn/T/pip-build-env-q96r4rwc/overlay/lib/python3.13/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License

In [12]:
import boto3
import pandas as pd
import json
import numpy as np
import pyarrow
import time
import os
import numpy as np
import pyarrow
import traceback
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain.embeddings import BedrockEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
#import dbconnection
import psycopg2
from psycopg2 import OperationalError
from dotenv import load_dotenv

# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

In [2]:
def limit_string_size(x, max_chars=2048):
    # Check if the input is a string
    if isinstance(x, str):
        return x[:max_chars]
    else:
        return x

In [3]:
def clean_value(value):
    value_str = str(value)
    cleaned_value = ''.join(char for char in value_str if char.isalnum() or char.isspace())
    return cleaned_value

In [4]:
def print_top_values(list_stuff: list, num_items: int) -> None:
    i=0
    for item in list_stuff:
        i=i+1
        if i>num_items:
            return None
        print(item)

In [5]:
# send in an array size of one and only return the 0th element
def generate_cohere_vector_embedding(text_data):
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    trunc_data = limit_string_size(text_data)
    # Create the JSON payload for the request
    json_params = {
            'texts': [trunc_data],
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(np.array(response['embeddings'][0]))


In [6]:
# Let's generate a dense vector using Amazon Titan with LangChain
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))



In [7]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



In [8]:
# Let's generate a dense vector using Amazon Titan without using a np.array as a return value
def generate_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    #Note pgvector does not want a np.array as out manual method
    return(embedding)

## Database info

In [9]:
# Let's review our IP address in case we haev any issues accessing the DB
from requests import get

ip = get('https://api.ipify.org').content.decode('utf8')
print('My public IP address is: {}'.format(ip))

My public IP address is: 129.65.145.216


In [ ]:
MY_USERNAME = "wkapner@calpoly.edu"

In [10]:
def insert_record_into_db(table_name, record, conn):

    # Dynamically generates and executes an INSERT SQL statement for PostgreSQL, handling
    # special data types like datetime objects and arrays directly.
    
    # Args:
    # - table_name (str): The name of the table into which the record will be inserted.
    # - data (dict): A dictionary representing the record to be inserted, where keys are column names
    #                  and values can include native PostgreSQL types like datetime and arrays.
    # - conn (psycopg2.connection): A psycopg2 connection object.
    
    # Generate column names and placeholders
    columns = ', '.join(record.keys())
    placeholders = ', '.join(['%s'] * len(record))  # PostgreSQL uses %s as placeholder

    # Create the INSERT INTO statement
    sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders})'
    # print("columns=", columns)
    # print("placeholders=", placeholders)
    # print(sql)
    try:
        cur = conn.cursor()
        
        cur.execute(sql, tuple(record.values()))
        conn.commit()
        print("Record inserted successfully.")
    except psycopg2.Error as e:
        print("An add record error occurred:", e)

In [ ]:
# Let's load a staff report for the city of SLO that contains 10 pages of data.  We will chunk the report into 1024 character chunks
# with a 256 character overlap.  We will insert each chunk into the DB with embeddings for similarity search.
with open("/Users/williamkapner/Documents/MSBA/GSB570AI/StudyDropshipping-ChatBot/video_chunks.json", "r") as file:
    text = file.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,  # Adjust the chunk size as needed
    chunk_overlap=256,  # Adjust the overlap between chunks as needed
)

# Split the text into chunks
chunks = text_splitter.split_text(text)

# Now for each chunk open a connection to our DB and insert a record into our generic schema

conn = dbconnection.open_connection_to_db()
try:
    # create a dataframe with new chunk raw text
    for chunk in chunks:
        data_record = {}
        v_embed = generate_vector_embedding(chunk)
        data_record["username"] = MY_USERNAME
        data_record["embedding"] = chunk
        data_record["content"] = ""
        data_record["source"] = ""
        data_record["textattribute4"] = ""
        data_record["textattribute5"] = ""
        data_record["textembedding1"] = v_embed
        insert_record_into_db("rag", data_record, conn)        
        
except Exception as e:
    print(f"An error occurred while inserting records: {e}")
finally:
    if conn:
        conn.close()

NameError: name 'dbconnection' is not defined

In [ ]:
def view_my_data(conn):
    try:
        cur = conn.cursor()

        # SQL statement to delete rows where username is 'bob'
        sql = f"SELECT textattribute1, textattribute2, textattribute3, textattribute4, textattribute5 FROM public.rag WHERE username = '{MY_USERNAME}'"
    
        # Execute the SQL statement
        cur.execute(sql)
        rows = cur.fetchall()

        # grab the cosine scores so we can compute Z score for narrow article selection
        for row in rows:
            print(row[0], row[1], row[2], row[3], row[4])
    
        

        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()
    

In [ ]:
conn = dbconnection.open_connection_to_db()
view_my_data(conn)

In [2]:
import sqlite3
from langchain.text_splitter import RecursiveCharacterTextSplitter

# You can use your actual embedding generator here
def generate_vector_embedding(text):
    # Replace with your actual embedding model
    # For example, return list of floats or bytes
    return text.encode("utf-8")[:512]  # Dummy placeholder for demonstration

MY_USERNAME = "wkapner@calpoly.edu"  # Replace with your actual username

# 1. Load text from JSON file
with open("/Users/williamkapner/Documents/MSBA/GSB570AI/StudyDropshipping-ChatBot/video_chunks.json", "r") as file:
    text = file.read()

# 2. Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=256,
)
chunks = text_splitter.split_text(text)

# 3. Define lightweight database connection and insert functions

def open_connection_to_db(db_path="my_chunks.db"):
    return sqlite3.connect(db_path)

def insert_record_into_db(table_name, data, conn):
    cursor = conn.cursor()

    # Create the table if it doesn't exist
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            username TEXT,
            textattribute1 TEXT,
            textattribute2 TEXT,
            textattribute3 TEXT,
            textattribute4 TEXT,
            textattribute5 TEXT,
            textembedding1 BLOB
        )
    """)

    # Insert the data record
    cursor.execute(f"""
        INSERT INTO {table_name} (
            username, textattribute1, textattribute2,
            textattribute3, textattribute4, textattribute5, textembedding1
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        data["username"],
        data["textattribute1"],
        data["textattribute2"],
        data["textattribute3"],
        data["textattribute4"],
        data["textattribute5"],
        data["textembedding1"]
    ))

    conn.commit()

# 4. Process and insert each chunk
conn = open_connection_to_db()

try:
    for chunk in chunks:
        data_record = {
            "username": MY_USERNAME,
            "textattribute1": chunk,
            "textattribute2": "",
            "textattribute3": "",
            "textattribute4": "",
            "textattribute5": "",
            "textembedding1": generate_vector_embedding(chunk)  # Replace with real vector
        }
        insert_record_into_db("rag", data_record, conn)

except Exception as e:
    print(f"An error occurred while inserting records: {e}")

finally:
    conn.close()


In [ ]:
def calculate_zscores(cosine_scores):
    zscores = []
    # Calculate the mean of the sample points
    mean = np.mean(cosine_scores)
    # Calculate the standard deviation of the sample points
    std_deviation = np.std(cosine_scores, ddof=1)  # ddof=1 for sample standard deviation
    # Calculate the z-scores for each sample point
    z_scores = [(x - mean) / std_deviation for x in cosine_scores]

    return z_scores

In [ ]:
def run_similarity_search_pgvector(question, embedded_text, conn):
    # Cosine similarity
    #1-(textembedding1 <=> ('{embedded_text}')) as cosine_similar  \
    # Euclidean distance
    #textembedding1 <-> ('{embedded_text}') as euclidean_distance  \
    sql = f"SELECT textattribute1, textattribute2, textattribute3, textattribute4, textattribute5, \
                1-(textembedding1 <=> ('{embedded_text}')) as cosine_similar  \
                FROM public.rag \
                WHERE username = '{MY_USERNAME}' \
                ORDER BY cosine_similar DESC \
                LIMIT 50"
    #print(sql)
    cosine_scores = []
    try:
        article_text = ""
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()

        # grab the cosine scores so we can compute Z score for narrow article selection
        # need all scores so we can calc Z
        for row in rows:
            #print(row[5])
            cosine_scores.append(row[5])
            
        z_scores = calculate_zscores(cosine_scores)
        answer = "Unknown"
        article_text = ""
        zscore_index = 0
        first_z_score = z_scores[0]
        for row in rows:
            if(first_z_score/2)<z_scores[zscore_index]:
                print(f"Using chunk with a cosine match of: {row[5]} and Z-score of: {z_scores[zscore_index]}")
                article_text = article_text + row[0] + "\n"
            zscore_index += 1
        #print(article_text)
        answer = best_answer(article_text, question)
        #Close cursor and connection
        cur.close()
        return answer
        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()
        
  

In [ ]:
import json
import pymysql
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Dummy embedding function — replace with real one (e.g. Bedrock, OpenAI)
def generate_vector_embedding(text):
    return [round((i % 7) * 0.1234, 4) for i in range(128)]  # Simulated 128-d vector

# Convert list of floats to string for MySQL storage
def serialize_embedding(vector):
    return ",".join(map(str, vector))

# Your Aurora MySQL credentials
DB_CONFIG = {
    "host": "your-db.cluster-xyz.us-west-2.rds.amazonaws.com",
    "user": "your_user",
    "password": "your_password",
    "database": "llm_db",
    "port": 3306
}

# Load the video/staff report content
with open("/Users/williamkapner/Documents/MSBA/GSB570AI/StudyDropshipping-ChatBot/video_chunks.json", "r") as file:
    text = file.read()

# Chunk it
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
chunks = splitter.split_text(text)

# Connect to MySQL
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS rag_chunks (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(255),
    chunk_text TEXT,
    embedding TEXT
);
""")
conn.commit()

# Insert chunks
MY_USERNAME = "willkapner"

for chunk in chunks:
    embedding = generate_vector_embedding(chunk)
    serialized = serialize_embedding(embedding)
    cursor.execute(
        "INSERT INTO rag_chunks (username, chunk_text, embedding) VALUES (%s, %s, %s);",
        (MY_USERNAME, chunk, serialized)
    )

conn.commit()
cursor.close()
conn.close()
